In [1]:
import numpy as np
import pandas as pd
import urllib
import bs4
import selenium
import re
import time
# selenium
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.by import By
#from selenium import webdriver

In [2]:
links = ['20190428', '20190331', '20190303', '20190203', '20190106', '20181209', '20181111', '20181014', '20180916', '20180819']

In [3]:
links

['20190428',
 '20190331',
 '20190303',
 '20190203',
 '20190106',
 '20181209',
 '20181111',
 '20181014',
 '20180916',
 '20180819']

In [4]:
def fetch_website(links):
    urls = []
    url = 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/'
    urls.append(url)
    url += 'sp'
    for i, l in  enumerate(links):
        url += '-d-' + links[i]
        urls.append(url)
    return urls

In [5]:
urls = fetch_website(links)

In [6]:
urls

['https://www.msn.com/en-us/sports/soccer/la-liga/scores/',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209-d-20181111',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209-d-20181111-d-20181014',
 'https://www.msn.com/en-us/sports/soccer/la-lig

In [7]:
def get_stats(stats):
    stat_list = ['BALL', 'SHOTS', 'FOULS', 'CORNERS', 'SAVES', 'OFFSIDES', 'YELLOW', 'RED']
    len_stats = len(stats)
    home_shots_og = None
    away_shots_og = None
    home_saves = None
    away_saves = None
    home_offsides = None
    away_offsides = None
    home_poss = None
    away_poss = None
    home_fouls = None
    away_fouls = None
    home_shots = None
    away_shots = None
    home_corners = None
    away_corners = None
    home_yellow = None
    away_yellow = None
    home_red = None
    away_red = None
    for i in range(len_stats):
        if stats[i][1] in stat_list:
            if stats[i][1] == 'BALL':
                home_poss = stats[i][0]
                away_poss = stats[i][-1]
            if stats[i][1] == 'FOULS':
                home_fouls = stats[i][0]
                away_fouls = stats[i][-1]
            if stats[i][1] == 'CORNERS':
                home_corners = stats[i][0]
                away_corners = stats[i][-1]
            if stats[i][1] == 'SAVES':
                home_saves = stats[i][0]
                away_saves = stats[i][-1]
            if stats[i][1] == 'OFFSIDES':
                home_offsides = stats[i][0]
                away_offsides = stats[i][-1]
            if stats[i][1] == 'YELLOW':
                home_yellow = stats[i][0]
                away_yellow = stats[i][-1]
            if stats[i][1] == 'RED':
                home_red = stats[i][0]
                away_red = stats[i][-1]
            if stats[i][1] == 'SHOTS':
                if len(stats[i]) == 3:
                    home_shots = stats[i][0]
                    away_shots = stats[i][-1]
                else:
                    home_shots_og = stats[i][0]
                    away_shots_og = stats[i][-1]
    return(home_poss, away_poss, home_shots, away_shots, home_shots_og, away_shots_og, home_fouls, away_fouls, home_corners, 
           away_corners, home_saves, away_saves, home_offsides, away_offsides, home_yellow, away_yellow, home_red, away_red)

In [30]:
def get_score_count_perhalf(score_link):
    result_link = score_link.find('div', {'id': 'main'}).find('div', {'class': 'teamevent'})
    home_team_link = result_link.find('div', {'class': 'hometeamevent'}).findAll('span', {'class': 'scoremins'})
    home_team_scores = []
    home_team_scores_firsthalf = 0
    home_team_scores_secondhalf = 0
    for h in home_team_link:
        score = h.text.strip().split("'")[0]
        if int(score) <= 45:
            home_team_scores_firsthalf += 1
        if int(score) > 45:
            home_team_scores_secondhalf += 1
        home_team_scores.append(score)

    away_team_link = result_link.find('div', {'class': 'visitingteamevent'}).findAll('span', {'class': 'scoremins'})
    away_team_scores = []
    away_team_scores_firsthalf = 0
    away_team_scores_secondhalf = 0
    for a in away_team_link:
        score = a.text.strip().split("'")[0]
        if int(score) <= 45:
            away_team_scores_firsthalf += 1
        if int(score) > 45:
            away_team_scores_secondhalf += 1
        away_team_scores.append(score)
        
    return(home_team_scores_firsthalf, home_team_scores_secondhalf, away_team_scores_firsthalf, away_team_scores_secondhalf)

In [31]:
def get_html(urls):
    teams = []
    for url in urls:
        html = urllib.request.urlopen(url).read()
        raw = bs4.BeautifulSoup(html, 'lxml')
        results = raw.find('div', {'class': 'sectionsgroup'}).findAll('tbody', {'class': ['even rowlink', 'odd rowlink']})
        for r in results:
            home_team = r.find('td', {'class': 'teamname teaminline alignright size23'}).text.strip()
            away_team = r.find('td', {'class': 'teamname size23'}).text.strip()
            match_date = r.find('td', {'class': 'groupingcolumn paddingleft'}).find('div', {'class': 'matchdate'}).text.strip()
            scores = r.findAll('td', {'class': 'teamscore'})
            
            #for s in scores:
                #home_team_score = s.text.strip()
            
            home_team_score = scores[0].text.strip()
            away_team_score = scores[1].text.strip()

            data_link = 'https://www.msn.com' + r.find('a').get('href')
            
            try:
                html_link = urllib.request.urlopen(data_link)
                raw_link = bs4.BeautifulSoup(html_link, 'lxml')
                result_link = raw_link.find('div', {'id': 'main'}).find('div', {'id': 'matchupgamestatsmodule'}).findAll('div', {'class': 'statsitem'})
            except urllib.error.HTTPError:
                # check this at the end. hard coded link
                html_link = urllib.request.urlopen('https://www.msn.com/en-us/sports/soccer/la-liga/eibar-v-barcelona/game-center/sp-id-80402000001009693')
                raw_link = bs4.BeautifulSoup(html_link, 'lxml')
                result_link = raw_link.find('div', {'id': 'main'}).find('div', {'id': 'matchupgamestatsmodule'}).findAll('div', {'class': 'statsitem'})
            stats = []
            chf, chs, caf, cas = get_score_count_perhalf(raw_link)
            for rl in result_link:
                stats.append(rl.text.split())
            '''home_team_posession = stats[0][0]
            away_team_posession = stats[0][-1]
            home_team_shots = stats[1][0]
            away_team_shots = stats[1][-1]
            home_team_shots_on_goal = stats[2][0]
            away_team_shots_on_goal = stats[2][-1]
            home_team_fouls = stats[3][0]
            away_team_fouls = stats[3][-1]
            home_team_corners = stats[4][0]
            away_team_corners = stats[4][-1]
            home_team_saves = stats[5][0]
            away_team_saves = stats[5][-1]
            home_team_y_cards = stats[6][0]
            away_team_y_cards = stats[6][-1]
            try:
                home_team_r_cards = stats[7][0]
            except IndexError:
                home_team_r_cards = 0
            try:
                away_team_r_cards = stats[7][-1]
            except IndexError:
                away_team_r_cards = 0'''
            res = get_stats(stats)
            teams.append([match_date.split(',')[0], match_date.split(',')[1][1:], home_team, away_team,
                          home_team_score, away_team_score, chf, chs, caf, cas, data_link, res[0], res[1]])
        time.sleep(5)
        break
    return teams

In [32]:
all_matches = get_html(urls)

In [33]:
all_matches

[['Sun',
  'May 19',
  'Eibar',
  'Barça',
  '2',
  '2',
  2,
  0,
  2,
  0,
  'https://www.msn.com/en-us/sports/soccer/la-liga/eibar-v-barcelona/game-center/sp-id-80402000001009693',
  '39',
  '61'],
 ['Sun',
  'May 19',
  'Real Madrid',
  'Betis',
  '0',
  '2',
  0,
  0,
  0,
  2,
  'https://www.msn.com/en-us/sports/soccer/la-liga/real-madrid-v-real-betis/game-center/sp-id-80402000001009694',
  '47',
  '53'],
 ['Sat',
  'May 18',
  'Alavés',
  'Girona',
  '2',
  '1',
  1,
  1,
  0,
  1,
  'https://www.msn.com/en-us/sports/soccer/la-liga/alav%c3%a9s-v-girona/game-center/sp-id-80402000001009686',
  '43',
  '57'],
 ['Sat',
  'May 18',
  'Celta',
  'Rayo',
  '2',
  '2',
  0,
  2,
  1,
  1,
  'https://www.msn.com/en-us/sports/soccer/la-liga/celta-de-vigo-v-rayo-vallecano/game-center/sp-id-80402000001009687',
  '45',
  '55'],
 ['Sat',
  'May 18',
  'Huesca',
  'Leganés',
  '2',
  '1',
  1,
  2,
  0,
  0,
  'https://www.msn.com/en-us/sports/soccer/la-liga/huesca-v-legan%c3%a9s/game-center/s

In [34]:
len(all_matches)

36